---
# MATH 434/534 Case Study: Social Networks
---

## Introduction

In this case study we will study the important question:

> How closely related are two people in a social network?

A social network, such as Facebook, can be modeled using an **undirected graph** $G = (V,E)$, where $V$ is a set of **vertices** (or **nodes**), and $E$ is a set of **edges**. For example, consider the following graph.

![text](https://upload.wikimedia.org/wikipedia/commons/5/5b/6n-graf.svg)

This graph has vertex set

$$
V = \{1,2,3,4,5,6\}
$$

and edge set

$$
E = \{\{1,2\}, \{1,5\}, \{2,3\}, \{2,5\}, \{3,4\}, \{4,5\}, \{4,6\}\}.
$$

On the Facebook graph, individuals are represented by nodes, and two individuals are connected by an edge if they are friends on Facebook. See the Wikipedia page on [Graph theory](https://en.wikipedia.org/wiki/Graph_theory) for more about graphs.

## Shortest-path distance in a social network

One measure of closeness between two people in a social network could be the length of the **shortest path** between them. In the above graph, the shortest path from node $1$ to node $6$ is the path $(1, 5, 4, 6)$, which has length $3$; thus, we could say that the distance between nodes $1$ and $6$ is $3$.

---
### Exercise 1

Complete the following shortest-path distance matrix for the above graph.

|  $d_{ij}$     | 1 | 2 | 3 | 4 | 5 | 6 |
|---    |:-:|:-:|:-:|:-:|:-:|:-:|
| **1** | 0 |   |   |   |   | 3 |
| **2** |   | 0 |   |   |   |   |
| **3** |   |   | 0 |   |   |   |
| **4** |   |   |   | 0 |   |   |
| **5** |   |   |   |   | 0 |   |
| **6** | 3 |   |   |   |   | 0 |

---

Although this is a simple way to measure the distance between individuals in a social network, it is not entirely appropriate. For example, both node $3$ and node $6$ have a shortest-path distance of $2$ from node $5$, yet we would all agree that node $3$ is "closer" to node $5$ since $3$ and $5$ have two friends in common, but $6$ and $5$ only have one friend in common.

## Resistance distance

Intuition tells us that our closeness measurement should obey the following rule:

> Two individuals are closer if they have more friends in common.

We will see **resistance distance** follows this rule much better than the shortest-path distance.

The notion of [resistance distance](https://en.wikipedia.org/wiki/Resistance_distance) had its origin in the study of electrical networks by [Klein and Randić (1993)](http://link.springer.com/article/10.1007%2FBF01164627) and is defined as the resistance between two nodes $i$ and $j$ in an electrical network, assuming $1 \Omega$ of resistance on each edge. [Bozzo and Franceschet (2013)](http://www.sciencedirect.com/science/article/pii/S0378873313000488) showed that resistance distance matches the information-theoretic distance measure among nodes in a network that was proposed by [Stephenson and Zelen (1989)](http://www.sciencedirect.com/science/article/pii/0378873389900166).

Before we can define the resistance distance, we need to first introduce some more notions from graph theory.

## Adjacency matrix

An undirected graph $G = (V, E)$ with $n$ nodes can be represented using an $n \times n$ symmetric matrix $A$ which is called the **adjacency matrix** of the graph. The entries of $A$ are defined as

$$
A_{ij} = 
\begin{cases}
1, & \text{if $\{i,j\} \in E$},\\
0, & \text{otherwise}.
\end{cases}
$$

---
### Exercise 2

Write down the adjacency matrix of the following graph.

![text](https://upload.wikimedia.org/wikipedia/commons/5/5b/6n-graf.svg "A graph with 6 nodes and 7 edges.")

---

## Laplacian matrix

The **Laplacian matrix** of an undirected graph $G = (V, E)$ with $n$ nodes is the $n \times n$ symmetric matrix $L$ defined by

$$
L = \mathrm{Diag}(d) - A,
$$

where: 

- $A$ is the adjacency matrix of $G$; 
- $d$ is the vector whose $i^\text{th}$ entry is the **degree** of node $i$ (i.e., the number of edges attached to node $i$);
- $\mathrm{Diag}(d)$ is the diagonal matrix having the vector $d$ along its diagonal.

---
### Exercise 3

1. Prove that $d = Ae$ and $Le = 0$, where $e$ is the vector of all ones.

2. Write down the Laplacian matrix of the following graph.

    ![text](https://upload.wikimedia.org/wikipedia/commons/5/5b/6n-graf.svg "A graph with 6 nodes and 7 edges.")
   
3. Write a Julia function called `laplacian` that takes an adjacency matrix $A$ and returns the corresponding Laplacian matrix $L$.

    ```julia
    L = laplacian(A)
    ```
    
    Test your function using the graph from part 2.
---

## Graph connectivity and the Laplacian matrix

A graph $G$ is called **connected** if any two nodes in $G$ are connected by some path. The above graph is clearly connected, but if we remove edges $\{3,4\}$ and $\{4,5\}$, then the graph will no longer be connected. 

It can be shown that $G$ is connected if and only if the **nullspace** of its Laplacian matrix 

$$
\mathrm{null}(L) = \{ x \mid Lx = 0 \}
$$

is equal to 

$$
\mathrm{span}\{e\} = \{ \lambda e \mid \lambda \in \mathbb{R}\}.
$$

In other words, $G$ is connected if and only if the multiplicity of the eigenvalue $0$ is one.

---
### Exercise 4

1. Prove that 

    $$
    L = \sum_{\{i,j\} \in E} (e_i - e_j)(e_i - e_j)^T
    $$

    where $e_i$ is the vector of length $n$ that is one in the $i^\mathrm{th}$ entry, and is zero in every other entry (i.e., $e_i$ is the $i^\mathrm{th}$ column of the $n \times n$ identity matrix).
    
2. Use the relationship from part 1 to prove that the Laplacian matrix is **positive semidefinite**; that is, prove that $x^T L x \geq 0$ for all vectors $x$.

3. Let $G$ be a connected graph and $L$ its Laplacian matrix. Prove that $L + ee^T$ is **positive definite**. (Hint: Use the fact that $L$ is positive semidefinite and thus $L = P^TP$ for some $n \times n$ matrix $P$.)

---

## Definition of resistance distance

The resistance distance $r_{ij}$ between nodes $i$ and $j$ on a connected graph $G$ is defined as

$$
r_{ij} = \|e_i - e_j\|_M^2,
$$

where 

$$
\|x\|_M = \sqrt{x^T M x} \qquad \text{and} \qquad M = \left(L + ee^T\right)^{-1}.
$$

Since $L + ee^T$ is positive definite, the matrix $M$ is also positive definite, which implies that $\| \cdot \|_M$ is a valid vector norm.

---
### Exercise 5

1. Prove that 

    $$
    r_{ij} = \|U^{-T}(e_i - e_j)\|_2^2
    $$

    where $U$ is the Cholesky factor of the positive definite matrix $L + ee^T$. Note that $U^{-T} = \left(U^T\right)^{-1} = \left(U^{-1}\right)^T$.

2. Compute $L + ee^T$ for the following graph and verify that it is positive definite by computing its Cholesky factor $U$.

    ![text](https://upload.wikimedia.org/wikipedia/commons/5/5b/6n-graf.svg "A graph with 6 nodes and 7 edges.")

3. Verify that node $5$ is closer to node $3$ than to node $6$ by computing the resistance distances $r_{35}$ and $r_{56}$ using the formula from part 1. To compute the vector $z = U^{-T}(e_i - e_j)$ efficiently, you should use backward substitution to solve the **lower-triangular** system $U^Tz = b$, where $b = e_i - e_j$; this is done in Julia using the command `z = U'\b`. Do not use the Julia command `z = inv(U')*b` since it is less efficient and less accurate.

4. Write a Julia function called `resistance` that returns the resistance distance $r_{ij}$ given the Cholesky factor $U$ of $L + ee^T$ and nodes $i$ and $j$.

    ```julia
    rij = resistance(U, i, j)
    ```

---

## A larger social network

For a larger example, we will now work with an author collaboration network on the [arXiv.org e-Print archive](http://arxiv.org/) in the area of *General Relativity and Quantum Cosmology*. The nodes of the graph will be authors of papers in this area and two authors will be connected by an edge if they have collaborated on a paper together.

The adjacency matrix for this graph, [ca-GrQc](http://www.cise.ufl.edu/research/sparse/matrices/SNAP/ca-GrQc.html), is available from the [The University of Florida Sparse Matrix Collection](https://www.cise.ufl.edu/research/sparse/matrices/). This is a large graph having 5242 nodes and 14484 edges.

However, the [ca-GrQc](http://www.cise.ufl.edu/research/sparse/matrices/SNAP/ca-GrQc.html) graph is not connected; for example, there are authors who have not collaborated with anyone else. We will instead work with the **largest connected component** of the graph, which has 4158 nodes and 13422 edges and can be visualized as follows.

![text](http://yifanhu.net/GALLERY/GRAPHS/GIF_SMALL/SNAP@ca-GrQc.gif)


---
### Exercise 6

1. Download the MATLAB matrix file [Graph.mat.zip](https://www.dropbox.com/s/bqspcdw2shv2xf7/Graph.mat.zip?dl=1) and unzip it to obtain the file `Graph.mat`. 

2. Use the Julia [MAT package](https://github.com/simonster/MAT.jl) to read the adjacency matrix $A$ stored in `Graph.mat`.

3. Compute the Laplacian matrix $L$.

4. Verify that $L + ee^T$ is positive definite by computing its Cholesky factor $U$.

5. Compute resistance distance $r_{ij}$ between node $i = 500$ and node $j = 3000$.

---

## Decreasing resistance distance  

We would like to verify that the resistance distance between two people in a social network decreases as one person becomes friends with the other person's friends.

We will do this by adding connections to the graph. We will make node $i$ friends with the friends of node $j$, one at a time, and compute the new resistance distance $r_{ij}$ each time.

Suppose node $k$ is a friend of node $j$. To make $i$ and $k$ friends, we simply need to make `A[i,k] = 1` and `A[k,i] = 1`.

---
### Exercise 7

1. Write a Julia function `change_resistance` that takes the adjacency matrix $A$, node $i$, and node $j$, and returns a vector $r$ that contains the value of $r_{ij}$ after node $i$ is connected to each friend of $j$, one at a time.

    ```julia
    r = change_resistance(A, i, j)
    ```

2. Test your function with $i = 500$ and $j = 3000$. Plot the vector $r$ to visualize how $r_{ij}$ decreases as nodes $i$ and $j$ become more closely connected.

---

## Improving efficiency using rank-one updates

You may have noticed that your implementation of `change_resistance` is slow since you needed to compute a new Cholesky decomposition of $L + ee^T$ each time a new edge was added to the graph.

However, the matrix $L + ee^T$ does not change much when a new edge is added to the graph. If we add the edge $\{i,k\}$ to the graph $G$, this corresponds to changing the $\{i,k\}$ and $\{k,i\}$ entries of the adjacency matrix $A$ from zero to one. This, in turn, will change the $\{i,k\}$, $\{k,i\}$, $\{i,i\}$, and $\{k,k\}$ entries of $L$. Changing these four entries of $L$ can be conveniently represented as replacing $L$ with $L + xx^T$, for a special vector $x$.

The operation $L + xx^T$ is known as a **rank-one update** of $L$ since $xx^T$ is a matrix whose rank is one.

---
### Exercise 8

Let $G$ be a graph and $L$ be its Laplacian matrix. 

1. Find the special vector $x$ such that $L + xx^T$ is the Laplacian matrix of the graph that is obtained by adding edge $\{i,k\}$ to the graph $G$.

2. Let $B = L + ee^T$ and suppose its Cholesky decomposition is $B = U^TU$. Let $\hat{L} = L + xx^T$ and $\hat{B} = \hat{L} + ee^T$. Show that

$$
\hat{B} = B + xx^T = 
\begin{bmatrix}
U \\ 
x^T \\
\end{bmatrix}^T
\begin{bmatrix}
U \\ 
x^T \\
\end{bmatrix}.
$$

---

## Updating the Cholesky factor

Given the Cholesky factorization $B = U^TU$ and a vector $x$, we want to efficiently compute the Cholesky factorization of $\hat{B} = B + xx^T$.

By Exercise 8, we can see that we *almost* have the Cholesky factorization of $\hat{B}$. However, the $(n+1) \times n$ matrix

$$
\begin{bmatrix}
U \\
x^T \\
\end{bmatrix}
$$

is not a square upper-triangular matrix with positive diagonal entries.

We have to find a way to fix this matrix so that it is upper-triangular with positive diagonal entries.

This can be done by finding an $(n+1) \times (n+1)$ orthogonal matrix $Q$ such that

$$
Q^T
\begin{bmatrix}
U \\
x^T \\
\end{bmatrix}
= 
\begin{bmatrix}
\hat{U} \\
0 \\
\end{bmatrix},
$$

where $\hat{U}$ is upper-triangular with positive diagonal entries. Then

$$
\hat{B} = \hat{U}^T \hat{U}.
$$

Thus, $\hat{U}$ would be the desired Cholesky factor of $\hat{B}$!

---
### Exercise 9

Show that if $Q$ is an $(n+1) \times (n+1)$ orthogonal matrix,

$$
\hat{B} =
\begin{bmatrix}
U \\ 
x^T \\
\end{bmatrix}^T
\begin{bmatrix}
U \\ 
x^T \\
\end{bmatrix},
\qquad
\text{and}
\qquad
Q^T
\begin{bmatrix}
U \\
x^T \\
\end{bmatrix}
= 
\begin{bmatrix}
\hat{U} \\
0 \\
\end{bmatrix},
$$

then $\hat{B} = \hat{U}^T \hat{U}$.

---

## Givens rotations

We will use Givens rotations to iteratively zero out the last row of 

$$
\begin{bmatrix}
U \\
x^T \\
\end{bmatrix}.
$$

Recall that $U = (u_{ij})$ is the Cholesky factor of the positive definite matrix $L + ee^T$, so $U$ is upper triangular and has _positive diagonal entries_.

Since $u_{11} > 0$, we can find an orthogonal rotation matrix

$$
Q = 
\begin{bmatrix}
c & -s \\
s &  c \\
\end{bmatrix}
$$

such that $c > 0$ and

$$
Q^T 
\begin{bmatrix}
u_{11} \\ x_1
\end{bmatrix}
=
\begin{bmatrix}
\sqrt{u_{11}^2 + x_1^2} \\ 0
\end{bmatrix}.
$$

(Note that $c^2 + s^2 = 1$ follows from $Q$ being orthogonal.)

We define the **Givens rotation matrix**

$$
Q_1 = 
\begin{bmatrix}
c & 0 & -s \\
0 & I_{n-1} & 0 \\
s & 0 & c \\
\end{bmatrix}.
$$

Then

$$
Q_1^T
\begin{bmatrix}
U \\
x^T \\
\end{bmatrix}
=
\begin{bmatrix}
c & 0 & s \\
0 & I_{n-1} & 0 \\
-s & 0 & c \\
\end{bmatrix}
\begin{bmatrix}
u_{11} & \bar{u}^T \\
0 & \bar{U} \\
x_1 & \bar{x}^T \\
\end{bmatrix}
=
\begin{bmatrix}
\sqrt{u_{11}^2 + x_1^2} & \hat{u}^T \\
0 & \bar{U} \\
0 & \hat{x}^T \\
\end{bmatrix}.
$$

Thus, we have zeroed out the first entry, $x_1$, in the last row. Notice that only the first and last rows have been modified.

We will proceed in a similar way to zero out $x_2,\ldots,x_n$.

---
### Exercise 10

1. Find the formulas for $c$ and $s$.

2. Find the formulas for $\hat{u}^T$ and $\hat{x}^T$.

---

## Cholesky update algorithm

Given an upper triangular matrix $U$ with positive diagonal entries and a vector $x$, the **Cholesky update algorithm** will overwrite $U$ with $\hat{U}$, and overwrite $x$ with zeros, such that

$$
\begin{bmatrix}
U \\ 
x^T \\
\end{bmatrix}^T
\begin{bmatrix}
U \\ 
x^T \\
\end{bmatrix}
= \hat{U}^T \hat{U},
$$

where $\hat{U}$ is upper triangular with positive diagonal entries.

The general description of the algorithm is as follows.

```julia
for j = 1:n
    # Use U[j,j] and x[j] to compute c and s.
    # Set U[j,j] = sqrt(U[j,j]^2 + x[j]^2).
    # Set x[j] = 0.
    # Use c and s to update U[j,j+1:n] and x[j+1:n].
end
```

---
### Exercise 11

1. Write a Julia function `chol_update!` that takes an $n \times n$ upper-triangular matrix $U$ with positive diagonal entries and a vector $x$ of length $n$, and overwrites $U$ with an $n \times n$ upper-triangular matrix $\hat{U}$ with postive diagonal entries such that $U^TU + xx^T = \hat{U}^T\hat{U}$.

    ```julia
    chol_update!(U, x)
    ```

2. Write a simple test using a random matrix and a random vector to verify that your `chol_update!` function is working correctly. Check your answer using Julia's `cholesky` function.
---

## The final stretch

We are now ready to significantly speed up the `change_resistance` function using our new `chol_update!` function.

---
### Exercise 12

1. Write a Julia function `change_resistance_fast` that takes the adjacency matrix $A$, node $i$, and node $j$, and returns a vector $r$ that contains the value of $r_{ij}$ after node $i$ is connected to each friend of $j$, one at a time.

    ```julia
    r = change_resistance_fast(A, i, j)
    ```
    
    This function should only use `cholesky` once to compute the initial Cholesky factor of $L + ee^T$. Afterwards, the Cholesky factor should be updated using your `chol_update!` function. The matrices $A$ and $L$ should not be directly updated.

2. Test your function with $i = 500$ and $j = 3000$. Verify that `change_resistance_fast` produces the same result as your `change_resistance` function. 

3. Use `@time` to compare the running time and memory allocation of `change_resistance` and `change_resistance_fast`.

4. Write a **short** conclusion of what you have learned in this case-study and a discussion of your results.

---